In [1]:
from tagger.core.mongo.models.order import Order, OrderCodeMap
from datetime import datetime
from tagger.models import Inventory
from tagger.models.inventory import InventoryStatus, ProductType
from tagger.core.label.receiving_label import make_receiving_label
from tagger.core.label.print_label import print_label
from bson import ObjectId
from tagger.core.mongo.models.product import Product
from tagger.core.mongo.models.alloff_product import AlloffProduct

import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [2]:
orders = Order.objects(orderstatus__nin=[
    "CREATED", "RECREATED", "PAYMENT_PENDING",
    "CANCEL_FINISHED", "DELIVERY_FINISHED", "CONFIRM_PAYMENT", "DELIVERY_STARTED"
])
order_map = {}

for order in orders:
    if order.orderstatus not in order_map:
        order_map[order.orderstatus] = []
    order_map[order.orderstatus].append(order)

In [5]:
def make_inventory(order: Order, is_return=False, needs_processing=None, target_item_index=None):
    if len(order.orders) > 1 and (target_item_index is None or target_item_index >= len(order.orders)):
        raise "Multiple items in the order... specify correct item index!"
    
    item = order.orders[0 if len(order.orders) == 1 else target_item_index]
    product = item.product if item.product is not None else item.alloffproduct
    product_type = ProductType.NORMAL_PRODUCT if item.product is not None else ProductType.TIMEDEAL_PRODUCT
    
    today = datetime.now().date()
    
    today_inventory_count = Inventory.objects.filter(created__gte=today).count()
    code_prefix = today.isoformat()[2:].replace("-", "")
    current_sequence = today_inventory_count + 1  # 0 inventories --> current is #1 (starts with 1)
    code = f"{code_prefix}-{str(current_sequence).zfill(3)}"
    if is_return:
        code += "-R"
    if needs_processing:
        code += "-NP"
    
    
    return Inventory.objects.create(
        code=code,
        status=InventoryStatus.PROCESSING_NEEDED if needs_processing else InventoryStatus.IN_STOCK,
        size=item.size,
        product_id=str(product._id),
        in_order_id=order.id,
        product_name=product.name,
        product_brand=product.brand.keyname,
        product_type=product_type
    )
    

In [6]:
inventory = make_inventory(Order.get("6173e41817fc2e5a1f9e9e67"), is_return=True)

# inventory = make_inventory(Order.get_by_code("uewke"), target_item_index=1)
print_label(make_receiving_label(inventory))

In [8]:
Order.get("6178d20c17fc2e5a1f9ea805")

[<Order: Order object>]

In [12]:
for i in Inventory.objects.all():
    product = (
        Product.objects(id=ObjectId(i.product_id)) if i.product_type == ProductType.NORMAL_PRODUCT 
        else AlloffProduct.objects(id=ObjectId(i.product_id))).first()
    
    i.product_brand = product.brand.keyname
    i.save()
    
               

In [16]:
Order.get_by_code("krjub").id

ObjectId('6180619517fc2e5a1f9eb42f')

In [8]:
print_label(make_receiving_label(Inventory.objects.get(id=37)))

'<?xml version="1.0" encoding="utf-8"?><DesktopLabel Version="1">  <DYMOLabel Version="3">    <Description>DYMO Label</Description>    <Orientation>Landscape</Orientation>    <LabelName>Addresss0722370</LabelName>    <InitialLength>0</InitialLength>    <BorderStyle>SolidLine</BorderStyle>    <DYMORect>      <DYMOPoint>        <X>0.23</X>        <Y>0.06</Y>      </DYMOPoint>      <Size>        <Width>3.21</Width>        <Height>0.9966666</Height>      </Size>    </DYMORect>    <BorderColor>      <SolidColorBrush>        <Color A="1" R="0" G="0" B="0"></Color>      </SolidColorBrush>    </BorderColor>    <BorderThickness>1</BorderThickness>    <Show_Border>False</Show_Border>    <DynamicLayoutManager>      <RotationBehavior>ClearObjects</RotationBehavior>      <LabelObjects>        <TextObject>          <Name>InventoryCode</Name>          <Brushes>            <BackgroundBrush>              <SolidColorBrush>                <Color A="0" R="0" G="0" B="0"></Color>              </SolidColorB

In [6]:
# For returns

inventory = make_inventory(Order.get("61812b1d26685ff051754ce4"), is_return=True)
print_label(make_receiving_label(inventory))

In [24]:
# print_label(make_receiving_label(Inventory.objects.get(id=143)))

In [19]:
Inventory.objects.filter(product_from_order_url=f"""https://office.alloff.co/orders/{Order.get("KTFKF").id}""")[0].created

datetime.datetime(2021, 11, 9, 4, 28, 27, 835328, tzinfo=<UTC>)

In [5]:
Order.get("euveb").id

ObjectId('6180fd6d26685ff051754c51')

In [23]:
print_label(make_receiving_label(Inventory.objects.get(pk=47)))

In [22]:
Inventory.objects.filter(code__startswith="201104").order_by('-code').first()

<Inventory: Inventory #30 [IN_STOCK] 볼륨 소매 니트 풀오버(민트) (201104-019)>

In [25]:
Inventory.objects.update(location="")

157

In [26]:
CUBE_ROOM = [
    "61813fbd26685ff051754d3c",
    "6185270426685ff051755530",
    "61812f0626685ff051754cfa",
    "6185735326685ff0517555d8",
    "61813fbd26685ff051754d3c",
    "618a64b026685ff051755da2",
    "6185735326685ff0517555d8",
    "61776a9017fc2e5a1f9ea5a4",
    "6171131b17fc2e5a1f9e9754",
    "616e1b5917fc2e5a1f9e91e6",
    "616e094917fc2e5a1f9e91bd",
    "616d707617fc2e5a1f9e907d",
    "617b49a017fc2e5a1f9ead04",
    "61701beb17fc2e5a1f9e9597",
    "6177e05817fc2e5a1f9ea668",
    "61715b9817fc2e5a1f9e9842",
]

CUBE_HANGER = [
    "617e246117fc2e5a1f9eb0e0",
    "6178c79a17fc2e5a1f9ea7c4",
    "6179f1d717fc2e5a1f9eaaa2",
    "61796a8017fc2e5a1f9eaa39",
    "6178fa2017fc2e5a1f9ea859",
    "6178d5d217fc2e5a1f9ea810",
    "6180fd6d26685ff051754c51",
    "6171f2c617fc2e5a1f9e997b",
]

SPARK_PLUS = [
    "6172b39017fc2e5a1f9e9c30",
    "6171f2c617fc2e5a1f9e997b",
    "6168ff9e17fc2e5a1f9e893c",
    "617a2e6917fc2e5a1f9eaae8",
    "616259aea789fd943b7ce0f6",
]

In [31]:
Inventory.objects.filter(in_order_id__in=CUBE_ROOM, status="IN_STOCK").update(location="큐브-박스")
Inventory.objects.filter(in_order_id__in=CUBE_HANGER, status="IN_STOCK").update(location="큐브-행거")
Inventory.objects.filter(in_order_id__in=SPARK_PLUS, status="IN_STOCK").update(location="본사")

2